In [1]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer , T5ForConditionalGeneration,Seq2SeqTrainer,Seq2SeqTrainingArguments
from transformers.models.t5.modeling_t5 import T5LayerSelfAttention , T5Attention

In [2]:
from TALib import TALib

In [3]:
ta_lib = TALib()

Map:   0%|          | 0/15159 [00:00<?, ? examples/s]

Map:   0%|          | 0/3790 [00:00<?, ? examples/s]

In [5]:
# tokenizer = AutoTokenizer.from_pretrained(TALib.TK_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(TALib.CHECKPOINT)

In [6]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [7]:
import torch

import torch.nn.utils.prune as prune

In [8]:
from transformers.models.t5.modeling_t5 import T5LayerSelfAttention, T5LayerCrossAttention, T5LayerFF

In [9]:
parameters_to_prune = {"self_attention" : [] , "cross_attention":[] , "ffn":[] , "lm_head":[]}
for name, module in model.named_modules():
    # print(name , type(module))
    if isinstance(module ,T5LayerSelfAttention ):
        # print("SelfAttention " , module)
        
        for name_2 , item in module.named_modules():
            if isinstance(item , torch.nn.Linear):
                parameters_to_prune["self_attention"].append((item , "weight"))
                
    if isinstance(module ,T5LayerCrossAttention ):
        # print("CrossAttention " , module)
        
        for name_2 , item in module.named_modules():
            if isinstance(item , torch.nn.Linear):
                parameters_to_prune["cross_attention"].append((item , "weight"))
                
    if isinstance(module ,T5LayerFF ):
        # print("FFN " , module)
        
        for name_2 , item in module.named_modules():
            if isinstance(item , torch.nn.Linear):
                parameters_to_prune["ffn"].append((item , "weight"))
                
    if isinstance(module ,torch.nn.Linear ) and name == "lm_head":
        parameters_to_prune["lm_head"].append((module , "weight"))
        
    # if isinstance(module, torch.nn.Linear):
    #     parameters_to_prune.append((module, "weight"))


In [10]:
parameters_to_prune

{'self_attention': [(Linear(in_features=512, out_features=512, bias=False),
   'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_

In [13]:
self_attention_prune_amount = 0.7
cross_attention_prune_amount = 0.5
ff_prune_amount = 0.8
lm_head_amount = 0.8

In [14]:
trainer = ta_lib.get_trainer(model=model )

In [16]:
def prune_model():
    prune.global_unstructured(
        parameters_to_prune["self_attention"],
        pruning_method=prune.L1Unstructured,
        amount=self_attention_prune_amount,
    )
    prune.global_unstructured(
        parameters_to_prune["cross_attention"],
        pruning_method=prune.L1Unstructured,
        amount=cross_attention_prune_amount,
    )
    prune.global_unstructured(
        parameters_to_prune["ffn"],
        pruning_method=prune.L1Unstructured,
        amount=ff_prune_amount,
    )
    prune.global_unstructured(
        parameters_to_prune["lm_head"],
        pruning_method=prune.L1Unstructured,
        amount=lm_head_amount,
    )
    return 

In [13]:
# prune_model()

In [17]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [18]:
TALib.show_param_ratio(model)

1.0

In [19]:
prune.global_unstructured(
    parameters_to_prune["self_attention"],
    pruning_method=prune.L1Unstructured,
    amount=self_attention_prune_amount,
)

In [20]:
TALib.show_param_ratio(model)

0.8544284701347351

In [21]:
trainer = ta_lib.get_trainer(model)

In [22]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 3.5016090869903564,
 'eval_rouge1': 0.0461,
 'eval_rouge2': 0.0138,
 'eval_rougeL': 0.0398,
 'eval_rougeLsum': 0.0398,
 'eval_gen_len': 18.9899,
 'eval_runtime': 738.3352,
 'eval_samples_per_second': 4.428,
 'eval_steps_per_second': 2.214}

In [23]:
trainer = ta_lib.get_trainer(model , num_train_epochs=15 , batch_size=5 ,output_dir="./output/pruning_v2")

In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.295400,2.070056,0.232900,0.182600,0.223300,0.223400,18.985200
2,2.094700,1.968425,0.236600,0.187900,0.227900,0.227900,18.991000
3,2.002000,1.915472,0.237700,0.189600,0.229500,0.229500,18.993900
4,1.946000,1.870970,0.238600,0.190900,0.230500,0.230500,18.999200
5,2.015200,1.841481,0.239800,0.192400,0.231400,0.231400,18.999500
6,2.093200,1.818560,0.240500,0.193100,0.232200,0.232200,18.998900
7,2.046600,1.800156,0.241300,0.194100,0.233100,0.233100,18.998700
8,2.076700,1.782918,0.241300,0.194200,0.233000,0.233000,18.998700
9,2.055200,1.771078,0.241400,0.194400,0.233100,0.233100,18.999500
10,1.731200,1.765161,0.241800,0.195100,0.233500,0.233500,18.999200


/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv

TrainOutput(global_step=45480, training_loss=2.025067795675474, metrics={'train_runtime': 18409.7044, 'train_samples_per_second': 12.351, 'train_steps_per_second': 2.47, 'total_flos': 6.154939105542144e+16, 'train_loss': 2.025067795675474, 'epoch': 15.0})

In [25]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

{'eval_loss': 1.7343477010726929,
 'eval_rouge1': 0.2409,
 'eval_rouge2': 0.1949,
 'eval_rougeL': 0.2327,
 'eval_rougeLsum': 0.2327,
 'eval_gen_len': 18.9982,
 'eval_runtime': 371.5103,
 'eval_samples_per_second': 8.799,
 'eval_steps_per_second': 1.76,
 'epoch': 15.0}

In [26]:
TALib.show_param_ratio(model)

0.8544284701347351

In [27]:
prune.global_unstructured(
    parameters_to_prune["cross_attention"],
    pruning_method=prune.L1Unstructured,
    amount=cross_attention_prune_amount,
)

In [28]:
TALib.show_param_ratio(model)

0.8024386763572693

In [29]:
trainer = ta_lib.get_trainer(model , num_train_epochs=15 , batch_size=5 ,output_dir="./output/pruning_v2")

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [30]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

{'eval_loss': 6.488123416900635,
 'eval_rouge1': 0.0024,
 'eval_rouge2': 0.0,
 'eval_rougeL': 0.0024,
 'eval_rougeLsum': 0.0024,
 'eval_gen_len': 19.0,
 'eval_runtime': 377.4686,
 'eval_samples_per_second': 8.66,
 'eval_steps_per_second': 1.733}

In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.753300,3.408405,0.161000,0.046500,0.132900,0.132900,18.967300
2,3.579600,3.320656,0.162400,0.049600,0.134900,0.134900,18.992300
3,3.455000,3.242739,0.167700,0.055700,0.139100,0.139200,18.994700
4,3.408400,3.182806,0.168200,0.059100,0.139900,0.139900,18.997600
5,3.280000,3.130174,0.173100,0.064800,0.145200,0.145200,18.998200
6,3.318600,3.082471,0.175000,0.069300,0.147500,0.147600,18.996800
7,3.345500,3.024870,0.185600,0.084700,0.157600,0.157600,18.999200
8,3.248600,2.978142,0.188000,0.089800,0.160700,0.160800,18.998900
9,3.345800,2.939803,0.189800,0.094200,0.164100,0.164200,18.997600
10,3.144100,2.907082,0.194100,0.100100,0.169500,0.169600,18.999500


/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv

TrainOutput(global_step=45480, training_loss=3.3524129954785136, metrics={'train_runtime': 18428.4413, 'train_samples_per_second': 12.339, 'train_steps_per_second': 2.468, 'total_flos': 6.154939105542144e+16, 'train_loss': 3.3524129954785136, 'epoch': 15.0})

In [32]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

{'eval_loss': 2.8593900203704834,
 'eval_rouge1': 0.1971,
 'eval_rouge2': 0.1097,
 'eval_rougeL': 0.1751,
 'eval_rougeLsum': 0.1751,
 'eval_gen_len': 18.9936,
 'eval_runtime': 369.5411,
 'eval_samples_per_second': 8.846,
 'eval_steps_per_second': 1.77,
 'epoch': 15.0}

In [33]:
prune.global_unstructured(
    parameters_to_prune["ffn"],
    pruning_method=prune.L1Unstructured,
    amount=ff_prune_amount,
)

In [34]:
TALib.show_param_ratio(model)

0.46970394253730774

In [35]:
trainer = ta_lib.get_trainer(model , num_train_epochs=15 , batch_size=5 ,output_dir="./output/pruning_v2")

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [36]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

{'eval_loss': 9.003531455993652,
 'eval_rouge1': 0.001,
 'eval_rouge2': 0.0,
 'eval_rougeL': 0.001,
 'eval_rougeLsum': 0.001,
 'eval_gen_len': 19.0,
 'eval_runtime': 378.8114,
 'eval_samples_per_second': 8.63,
 'eval_steps_per_second': 1.726}

In [37]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,4.765000,4.284475,0.079500,0.017100,0.073400,0.073300,18.989700
2,4.469300,4.044209,0.100400,0.027600,0.088500,0.088500,18.996600
3,4.263900,3.902002,0.119300,0.039300,0.102900,0.102900,18.991600
4,4.124200,3.799929,0.130000,0.047800,0.111600,0.111600,18.982300
5,4.026900,3.719667,0.142700,0.056900,0.121900,0.121900,18.989400
6,3.997400,3.657851,0.150000,0.062500,0.128400,0.128300,18.991600
7,4.026200,3.603822,0.155000,0.067000,0.133100,0.133200,18.992100
8,3.940800,3.561581,0.160000,0.070700,0.137500,0.137500,18.992900
9,4.024400,3.526291,0.162800,0.074600,0.140700,0.140800,18.992300
10,3.824700,3.497958,0.164300,0.076600,0.142300,0.142300,18.995500


/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv

TrainOutput(global_step=45480, training_loss=4.127628256211494, metrics={'train_runtime': 18569.2723, 'train_samples_per_second': 12.245, 'train_steps_per_second': 2.449, 'total_flos': 6.154939105542144e+16, 'train_loss': 4.127628256211494, 'epoch': 15.0})

In [38]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

{'eval_loss': 3.45656156539917,
 'eval_rouge1': 0.1674,
 'eval_rouge2': 0.0823,
 'eval_rougeL': 0.1468,
 'eval_rougeLsum': 0.1467,
 'eval_gen_len': 18.9939,
 'eval_runtime': 378.223,
 'eval_samples_per_second': 8.643,
 'eval_steps_per_second': 1.729,
 'epoch': 15.0}

In [39]:
prune.global_unstructured(
    parameters_to_prune["lm_head"],
    pruning_method=prune.L1Unstructured,
    amount=lm_head_amount,
)

In [40]:
TALib.show_param_ratio(model)

0.2522132098674774

In [41]:
trainer = ta_lib.get_trainer(model , num_train_epochs=30 , batch_size=5 ,output_dir="./output/pruning_v2")

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [42]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 7.87841272354126,
 'eval_rouge1': 0.0226,
 'eval_rouge2': 0.0015,
 'eval_rougeL': 0.0217,
 'eval_rougeLsum': 0.0217,
 'eval_gen_len': 18.7675,
 'eval_runtime': 416.9711,
 'eval_samples_per_second': 7.84,
 'eval_steps_per_second': 1.568}

In [43]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,6.418000,5.713407,0.064100,0.020000,0.056900,0.056900,18.942000
2,5.886300,5.259257,0.113200,0.047600,0.100200,0.100100,18.968100
3,5.597900,5.001430,0.134100,0.062400,0.118700,0.118700,19.000000
4,5.399200,4.822623,0.143300,0.069400,0.127500,0.127300,19.000000
5,5.205700,4.683297,0.149800,0.075100,0.133700,0.133600,18.989400
6,5.176300,4.573000,0.153800,0.079500,0.138100,0.138100,18.959600
7,5.164500,4.479920,0.154800,0.084700,0.140200,0.140200,18.964400
8,5.037000,4.401291,0.157100,0.088300,0.143100,0.143200,18.977300
9,5.081400,4.331436,0.159700,0.091400,0.145800,0.145900,18.972600
10,4.842900,4.267817,0.163200,0.094700,0.149100,0.149100,18.976300


/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv

TrainOutput(global_step=90960, training_loss=4.944763226412636, metrics={'train_runtime': 37909.0211, 'train_samples_per_second': 11.996, 'train_steps_per_second': 2.399, 'total_flos': 1.2309878211084288e+17, 'train_loss': 4.944763226412636, 'epoch': 30.0})

In [44]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

{'eval_loss': 3.8354501724243164,
 'eval_rouge1': 0.191,
 'eval_rouge2': 0.1212,
 'eval_rougeL': 0.1765,
 'eval_rougeLsum': 0.1765,
 'eval_gen_len': 18.9942,
 'eval_runtime': 400.9625,
 'eval_samples_per_second': 8.153,
 'eval_steps_per_second': 1.631,
 'epoch': 30.0}

In [45]:
TALib.show_param_ratio(model)

0.2522132098674774

In [46]:
results = trainer.predict(ta_lib.tokenized_billsum_test)

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [47]:
decoded_prediction = ta_lib.tokenizer.batch_decode(results[0], skip_special_tokens=True)

In [48]:
TALib.dump_to_kaggle_format(decoded_prediction , 'pruned_model_0.25_day_5_4.csv')

,ID,Predict
0,0,Amends the Water Resources Development Act of ...
1,1,Farm Home Fee Act of 1993 - Amends the Interna...
2,2,Memo Memo Memo Memo Act - Authorizes the Secre...
3,3,Small Business Tax Modernization Act of 2004 -...
4,4,Fair Accessing Investment Research Act of 2016...
...,...,...
3264,3264,Public Serpozment Act of 1995 - Amends the Off...
3265,3265,Animalsmanship of 2008 - Amends the Interior t...
3266,3266,Academic Student Student Student Student Stude...
3267,3267,National Forests Improvement Act of 2000 - Ame...


In [49]:
TALib.run_score(decoded_prediction , ta_lib.billsum_test)

0.13524281905366212

In [50]:
TALib.save_model(model , "output/pruning_success")

In [51]:
reload_model = TALib.load_model("output/pruning_success")

Some weights of the model checkpoint at output/pruning_success were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.0.SelfAttention.k.weight_mask', 'decoder.block.0.layer.0.SelfAttention.k.weight_orig', 'decoder.block.0.layer.0.SelfAttention.o.weight_mask', 'decoder.block.0.layer.0.SelfAttention.o.weight_orig', 'decoder.block.0.layer.0.SelfAttention.q.weight_mask', 'decoder.block.0.layer.0.SelfAttention.q.weight_orig', 'decoder.block.0.layer.0.SelfAttention.v.weight_mask', 'decoder.block.0.layer.0.SelfAttention.v.weight_orig', 'decoder.block.0.layer.1.EncDecAttention.k.weight_mask', 'decoder.block.0.layer.1.EncDecAttention.k.weight_orig', 'decoder.block.0.layer.1.EncDecAttention.o.weight_mask', 'decoder.block.0.layer.1.EncDecAttention.o.weight_orig', 'decoder.block.0.layer.1.EncDecAttention.q.weight_mask', 'decoder.block.0.layer.1.EncDecAttention.q.weight_orig', 'decoder.block.0.layer.1.EncDecAttention.v.weight_mask', 'decoder.block.0.layer.1.EncDecAttent

In [53]:
TALib.show_param_ratio(reload_model)

0.2522132098674774

In [55]:
trainer = ta_lib.get_trainer(reload_model , num_train_epochs=30 , batch_size=5 ,output_dir="./output/pruning_check")

In [56]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 3.8354501724243164,
 'eval_rouge1': 0.191,
 'eval_rouge2': 0.1212,
 'eval_rougeL': 0.1765,
 'eval_rougeLsum': 0.1765,
 'eval_gen_len': 18.9942,
 'eval_runtime': 383.5045,
 'eval_samples_per_second': 8.524,
 'eval_steps_per_second': 1.705}